In [2]:
import pandas as pd
import os
from jenga.corruptions import DuckDBCorruptionBrokenCharacters
import duckdb

In [3]:
df = pd.read_json("./data/small.json", lines=True)
df

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote
0,5,True,"11 19, 2014",A1QVBUH9E1V6I8,4639725183,Jamshed Mathur,No adverse comment.,Five Stars,1416355200,NaN
1,5,True,"10 13, 2016",A3GEOILWLK86XM,4639725183,itsjustme,Gift for college student.,Great product.,1476316800,NaN
2,5,True,"11 21, 2015",A32RD6L701BIGP,4639725183,Krystal Clifton,"If you like strong tea, this is for you. It mi...",Strong,1448064000,NaN
3,5,True,"08 12, 2015",A2UY1O1FBGKIE6,4639725183,U. Kane,Love the tea. The flavor is way better than th...,Great tea,1439337600,NaN
4,5,True,"05 28, 2015",A3QHVBQYDV7Z6U,4639725183,The Nana,I have searched everywhere until I browsed Ama...,This is the tea I remembered!,1432771200,NaN
...,...,...,...,...,...,...,...,...,...,...
112,5,True,"10 28, 2014",A3HKBY3DH96VVT,9742356831,Icydawg,as described and shipped on time tastey,Five Stars,1414454400,NaN
113,4,True,"10 1, 2014",A1FIOBU4XSMO56,9742356831,WRS,Good stuff. Would order again.,Very good.,1412121600,NaN
114,5,True,"09 22, 2014",A4EWRIA8CCRMQ,9742356831,Ann,yum,Five Stars,1411344000,NaN
115,4,True,"09 13, 2014",A9DIYBDBJDOJ,9742356831,Blue Dragonfly,I have Mae Ploy Yellow Curry and Panang Red Cu...,Very hot!,1410566400,13.0


In [ ]:
duckdb_corruption = DuckDBCorruptionBrokenCharacters("reviewText", 0.2)
duckdb_corruption.transform(df)

In [24]:
exceptions = """-_*@"""
homos = """˗৭Ȣ𝟕бƼᏎƷᒿlO`ɑЬϲԁе𝚏ɡհіϳ𝒌ⅼｍոорԛⲅѕ𝚝սѵԝ×уᴢ"""
filter_pattern = homos + """\\-_\\*@"""
filter_pattern = f"[\\w{filter_pattern}]+"
print(f"{exceptions}")
duckdb.sql(f"""
    WITH 
        words AS (
           SELECT 
            reviewerId,
            asin,
            flatten(   
                [   
                    regexp_extract_all(
                        ltrim(w, '{exceptions}' ),
                        '{filter_pattern}'
                    )
                    FOR w in 
                    regexp_split_to_array(reviewText, ' ') 
                ]
            ) as word
            FROM df
        ),
        delete_words AS (
            SELECT 
                reviewerId,
                asin,
                word[CAST(floor(len(word) * random()) AS int)] AS selected_word
            FROM words
        )
        

    SELECT 
        regexp_replace(df.reviewText, dw.selected_word, '') AS reviewText, 
        df.reviewText as oldText,
        dw.selected_word,
        df.asin, dw.asin    
    FROM delete_words dw
    JOIN df ON df.asin = dw.asin AND df.reviewerId = dw.reviewerId
    LIMIT 10
""").df()

-_*@


,reviewText,oldText,selected_word,asin,asin_1
0,adverse comment.,No adverse comment.,No,4639725183,4639725183
1,Gift college student.,Gift for college student.,for,4639725183,4639725183
2,"If you like strong tea, this is for you. migh...","If you like strong tea, this is for you. It mi...",It,4639725183,4639725183
3,Love the . The flavor is way better than the r...,Love the tea. The flavor is way better than th...,tea,4639725183,4639725183
4,I have searched everywhere until I browsed Ama...,I have searched everywhere until I browsed Ama...,awful,4639725183,4639725183
5,Tea made Lipton Yellow Label teabags is much ...,Tea made with Lipton Yellow Label teabags is m...,with,4639725183,4639725183
6,"I love tea! Okay, I'm not a high falutin' te...","I love this tea! Okay, I'm not a high falutin...",this,4639725183,4639725183
7,Discovered this tea at a local Med. Rest....a ...,Discovered this tea at a local Med. Rest....a ...,for,4639725183,4639725183
8,Well I bought this tea being in Malaysia for ...,Well I bought this tea after being in Malaysia...,after,4639725183,4639725183
9,We really like this tea. It is definitely dif...,We really like this tea. It is definitely dif...,regular,4639725183,4639725183
